In [1]:
from pathlib import Path

import numpy as np
from numba import njit, prange

import pandas as pd

In [2]:
df = pd.read_csv(Path('..', '..', '..', 'data', 'iris_csv.csv'))

for c in df.columns[0:4]:
    df[c] = (df[c]-df[c].mean())/df[c].std()

df['synth1'] = df['petallength']*df['petalwidth']
df['synth2'] = df['sepallength']*df['petallength']
df['synth3'] = df['sepallength']*df['petalwidth']

for name in df['class'].unique():
    df[f'label-{name}'] = df['class'].map(lambda x: 1 if x == name else 0)

In [3]:
np.random.seed(0)

setosa_idxs = np.arange(0, 50)
versicolor_idxs = np.arange(50, 100)
virginica_idxs = np.arange(100, 150)

p = np.random.permutation(np.arange(50))

setosa_train_idxs = setosa_idxs[p[0:10]]
setosa_test_idxs = setosa_idxs[p[10:]]

versicolor_train_idxs = versicolor_idxs[p[0:10]]
versicolor_test_idxs = versicolor_idxs[p[10:]]

virginica_train_idxs = virginica_idxs[p[0:10]]
virginica_test_idxs = virginica_idxs[p[10:]]

feature_columns = ['sepallength', 'sepalwidth', 'petallength', 'petalwidth']
label_columns = ['label-Iris-setosa', 'label-Iris-versicolor', 'label-Iris-virginica']

xTrain = np.vstack([
    df.iloc[setosa_train_idxs][feature_columns],
    df.iloc[versicolor_train_idxs][feature_columns],
    df.iloc[virginica_train_idxs][feature_columns]
])

yTrain = np.vstack([
    df.iloc[setosa_train_idxs][label_columns],
    df.iloc[versicolor_train_idxs][label_columns],
    df.iloc[virginica_train_idxs][label_columns]
])

xTest = np.vstack([
    df.iloc[setosa_test_idxs][feature_columns],
    df.iloc[versicolor_test_idxs][feature_columns],
    df.iloc[virginica_test_idxs][feature_columns]
])

yTest = np.vstack([
    df.iloc[setosa_test_idxs][label_columns],
    df.iloc[versicolor_test_idxs][label_columns],
    df.iloc[virginica_test_idxs][label_columns]
])

In [4]:

def softmax(x: np.ndarray) -> float:
    y = np.exp(x)
    return y/np.sum(y)


@njit(fastmath=True)
def softmax_jit(x: np.ndarray) -> float:
    y = np.exp(x)
    return y/np.sum(y)


@njit(fastmath=True)
def grads(xBatch: np.ndarray, yBatch: np.ndarray, w: np.ndarray, b: np.ndarray) -> tuple[np.ndarray]:
    n = xBatch.shape[0]

    dw = np.zeros(w.shape)
    db = np.zeros(b.shape)
    
    for i in prange(n):
        y = softmax_jit(xBatch[i] @ w + b)
        dw += np.outer(xBatch[i], y - yBatch[i])
        db += y - yBatch[i]
    
    return (dw, db)

class Perceptron:
    def __init__(self, nIn: int, nOut: int) -> None:
        self.nIn = nIn
        self.nOut = nOut
        self.w: np.ndarray = np.random.uniform(-1, 1, (nIn, nOut))
        self.b: np.ndarray = np.zeros((nOut))

    def predict(self, x:np.ndarray) -> np.ndarray:
        return softmax(x @ self.w + self.b)

    def train(self, xTrain: np.ndarray, yTrain: np.ndarray, lr, batch_size, max_iter) -> None:
        n = xTrain.shape[0]

        for i in range(max_iter):
            idxs = np.random.choice(a=np.arange(n), size=batch_size, replace=False)

            dw, db = grads(xTrain[idxs], yTrain[idxs], self.w, self.b)
            
            self.w -= lr*dw
            self.b -= lr*db

    def loss(self, x: np.ndarray, y: np.ndarray) -> float:        
        Ypred = np.array([self.predict(xi) for xi in x])
        h = -np.sum(y*np.log(Ypred), axis=1)
        return 1/x.shape[0] * np.sum(h)

In [5]:
nIn = 4
nOut = 3

learning_rate = 1e-2
batch_size = 30
max_iter = 1000

model = Perceptron(nIn, nOut)

print('untrained loss: ', model.loss(xTest, yTest).round(3))

model.train(
    xTrain,
    yTrain,
    learning_rate,
    batch_size,
    max_iter
)

print('trained loss: ', model.loss(xTest, yTest).round(3))

TP_count = 0
for x, y in zip(xTest, yTest):
    yPred = model.predict(x)
    TP_count += 1 if np.argmax(y) == np.argmax(yPred) else 0

accuracy = TP_count / xTest.shape[0]
print('accuracy: ', accuracy)

untrained loss:  0.868
trained loss:  0.158
accuracy:  0.95


In [6]:
for x,y in zip(xTest, yTest):
    print(y, model.predict(x).round(2))

[1 0 0] [1. 0. 0.]
[1 0 0] [1. 0. 0.]
[1 0 0] [1. 0. 0.]
[1 0 0] [0.99 0.01 0.  ]
[1 0 0] [0.99 0.01 0.  ]
[1 0 0] [1. 0. 0.]
[1 0 0] [1. 0. 0.]
[1 0 0] [0.99 0.01 0.  ]
[1 0 0] [1. 0. 0.]
[1 0 0] [1. 0. 0.]
[1 0 0] [1. 0. 0.]
[1 0 0] [1. 0. 0.]
[1 0 0] [1. 0. 0.]
[1 0 0] [1. 0. 0.]
[1 0 0] [1. 0. 0.]
[1 0 0] [0.98 0.02 0.  ]
[1 0 0] [1. 0. 0.]
[1 0 0] [1. 0. 0.]
[1 0 0] [1. 0. 0.]
[1 0 0] [0.98 0.02 0.  ]
[1 0 0] [1. 0. 0.]
[1 0 0] [1. 0. 0.]
[1 0 0] [1. 0. 0.]
[1 0 0] [1. 0. 0.]
[1 0 0] [0.99 0.01 0.  ]
[1 0 0] [1. 0. 0.]
[1 0 0] [0.99 0.01 0.  ]
[1 0 0] [1. 0. 0.]
[1 0 0] [1. 0. 0.]
[1 0 0] [0.99 0.01 0.  ]
[1 0 0] [0.99 0.01 0.  ]
[1 0 0] [1. 0. 0.]
[1 0 0] [1. 0. 0.]
[1 0 0] [0.99 0.01 0.  ]
[1 0 0] [1. 0. 0.]
[1 0 0] [1. 0. 0.]
[1 0 0] [1. 0. 0.]
[1 0 0] [1. 0. 0.]
[1 0 0] [1. 0. 0.]
[1 0 0] [1. 0. 0.]
[0 1 0] [0.   0.61 0.39]
[0 1 0] [0.01 0.42 0.58]
[0 1 0] [0.   0.99 0.01]
[0 1 0] [0.02 0.54 0.45]
[0 1 0] [0.   0.99 0.01]
[0 1 0] [0.16 0.84 0.  ]
[0 1 0] [0.04 0.96 0.01]
[0 1 